# GWAS Tutorial (Exercise Version): Data Exploration, QC & Association Analysis

This is the **exercise version** of the tutorial. Key code cells have been partially or fully replaced with stubs — your job is to fill them in.

All computation (Hail aggregations, QC functions, GWAS) is provided. You write the **visualisations** and **regression calls**.

> If you get stuck, compare with the [complete version](https://colab.research.google.com/github/gosborcz/winterschool-gwas-tutorial/blob/main/gwas_tutorial.ipynb).

## 1. Setup

In [ ]:
# Install Java (Hail's Spark backend) and Hail
!apt-get install -y openjdk-11-jdk-headless -q
!pip install hail -q

import hail as hl
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from pprint import pprint

hl.init()
from hail.plot import show
hl.plot.output_notebook()

import os
os.makedirs('data', exist_ok=True)
hl.utils.get_1kg('data/')

hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)
mt = hl.read_matrix_table('data/1kg.mt')

table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')
mt = mt.annotate_cols(pheno=table[mt.s])
print('Loaded: %d variants x %d samples' % mt.count())

## 2. Data Exploration

### The MatrixTable

Hail stores genomic data in a **MatrixTable** — a 2D structure where:

```
              Sample1  Sample2  Sample3  ...
 chr1:100 A/T   0/1      0/0      1/1
 chr1:200 G/C   0/0      0/1      0/1
    ...          ...      ...      ...
```

- **Rows** = variants (locus + alleles)
- **Columns** = samples (sample ID `s`)
- **Entries** = genotype calls (GT, DP, GQ, AD, PL)
- **Row fields** = variant INFO annotations
- **Column fields** = sample metadata

In [ ]:
mt.rows().select().show(5)

In [ ]:
mt.entry.take(3)

### Sample Annotations

In [ ]:
table.show(5, width=120)

### Phenotype & Population Overview

In [ ]:
pop_counts = mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))
pprint(pop_counts)

In [ ]:
pops = sorted(pop_counts.keys())
counts = [pop_counts[p] for p in pops]

fig, ax = plt.subplots(figsize=(8, 4))
# YOUR CODE HERE: bar chart of pops vs counts, label axes, annotate bars with counts


In [ ]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))

In [ ]:
# YOUR CODE HERE: plot the distribution of CaffeineConsumption
# Hint: use hl.plot.histogram(mt.pheno.CaffeineConsumption, bins=30, title=..., legend=...)


### Variant Exploration

In [ ]:
snp_counts = mt.aggregate_rows(
    hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1]))
)
print('Top 10 allele changes:')
for k, v in Counter(snp_counts).most_common(10):
    print('  %s -> %s: %d' % (k.ref, k.alt, v))

#### Allele Frequency Spectrum

Most variants are rare — an L-shaped AFS is the hallmark of healthy genetic data under neutral evolution.

In [ ]:
afs = mt.aggregate_rows(hl.agg.hist(mt.info.AF[0], 0, 1.0, 50))
edges, freq = afs.bin_edges, afs.bin_freq
mids = [(a + b) / 2 for a, b in zip(edges, edges[1:])]
width = (edges[1] - edges[0]) * 0.9

fig, ax = plt.subplots(figsize=(9, 4))
# YOUR CODE HERE: bar chart with mids on x-axis, freq on y-axis, label axes
# Hint: ax.bar(mids, freq, width=width, ...)


#### Variant Density by Chromosome

In [ ]:
chrom_counts = mt.aggregate_rows(hl.agg.counter(mt.locus.contig))

def chrom_sort_key(c):
    n = c.replace('chr', '')
    return (0, int(n)) if n.isdigit() else (1, n)

chroms = sorted(chrom_counts.keys(), key=chrom_sort_key)
cnts = [chrom_counts[c] for c in chroms]

fig, ax = plt.subplots(figsize=(13, 4))
# YOUR CODE HERE: bar chart of variants per chromosome
# Hint: use ax.bar(range(len(chroms)), cnts, ...) then ax.set_xticks + ax.set_xticklabels


#### Genotype Distribution

Because most variants are rare, the vast majority of entries should be homozygous reference (0/0).

In [ ]:
gt_counts = mt.aggregate_entries(hl.agg.counter(mt.GT.n_alt_alleles()))

labels = ['Hom Ref (0/0)', 'Het (0/1)', 'Hom Alt (1/1)']
values = [gt_counts.get(k, 0) for k in range(3)]
total = sum(values)
colors = ['#4e9af1', '#f4a261', '#e76f51']

fig, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 4))
# YOUR CODE HERE:
# ax: bar chart of labels vs values
# ax2: pie chart with autopct='%1.1f%%'
# Also print counts + percentages below the plot


---
**🔍 Exercise 1:** The bar chart above shows counts for the 5 super-populations. Modify it to show the breakdown by the more specific `Population` field. How many distinct populations are in this dataset?

> *Hint: change `mt.pheno.SuperPopulation` → `mt.pheno.Population` in the `agg.counter` call and re-run the bar chart.*

In [ ]:
# Exercise 1 — your code here


## 3. Quality Control

### Sample QC

`hl.sample_qc()` computes per-sample metrics:

| Metric | Meaning |
|---|---|
| `call_rate` | Fraction of non-missing genotype calls |
| `dp_stats.mean` | Mean read depth |
| `gq_stats.mean` | Mean genotype quality |

In [ ]:
mt = hl.sample_qc(mt)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.dp_stats.mean, range=(0, 30), bins=30,
                      title='Mean Read Depth per Sample', legend='Mean DP')
show(p)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(0.88, 1.0), bins=30,
                      title='Call Rate per Sample', legend='Call Rate')
show(p)

In [ ]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10, 70), bins=30,
                      title='Mean GQ per Sample', legend='Mean GQ')
show(p)

In [ ]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate,
                    xlabel='Mean DP', ylabel='Call Rate',
                    title='Mean Depth vs. Call Rate')
show(p)

In [ ]:
n_before = mt.count_cols()
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('Samples: %d → %d (removed %d)' % (n_before, mt.count_cols(), n_before - mt.count_cols()))

### Allelic Balance Filter

The **allelic balance (AB)** — fraction of reads supporting the alt allele — should be:
- ~0 for hom ref (0/0)
- ~0.5 for het (0/1)
- ~1 for hom alt (1/1)

In [ ]:
ab = mt.AD[1] / hl.sum(mt.AD)
filter_ab = (
    (mt.GT.is_hom_ref() & (ab <= 0.1)) |
    (mt.GT.is_het()     & (ab >= 0.25) & (ab <= 0.75)) |
    (mt.GT.is_hom_var() & (ab >= 0.9))
)
fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_ab))
print('Setting %.2f%% of entries to missing (failed AB filter).' % (fraction_filtered * 100))
mt = mt.filter_entries(filter_ab)

---
**🔍 Exercise 2:** The current het allelic balance window is **0.25–0.75**. What fraction of entries would *additionally* be removed if you tightened it to **0.30–0.70**?

> *Hint: compute `hl.agg.fraction(~filter_ab_strict)` using the stricter condition on `mt` before calling `filter_entries`.*

In [ ]:
# Exercise 2 — your code here


### Variant QC

`hl.variant_qc()` computes per-variant metrics:

| Metric | Meaning |
|---|---|
| `AF[1]` | Alternate allele frequency in this dataset |
| `call_rate` | Fraction of samples with a call |
| `p_value_hwe` | Hardy-Weinberg equilibrium test p-value |

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
af_hist = mt.aggregate_rows(hl.agg.hist(mt.variant_qc.AF[1], 0, 0.5, 50))
edges, freq = af_hist.bin_edges, af_hist.bin_freq
mids = [(a + b) / 2 for a, b in zip(edges, edges[1:])]
width = (edges[1] - edges[0]) * 0.9

fig, ax = plt.subplots(figsize=(9, 4))
# YOUR CODE HERE: bar chart of MAF distribution
# Add a vertical dashed red line at x=0.01 to mark the filter threshold
# Hint: ax.axvline(x=0.01, color='red', linestyle='--', label='AF = 1% filter')


In [ ]:
hwe_hist = mt.aggregate_rows(
    hl.agg.filter(hl.is_defined(mt.variant_qc.p_value_hwe),
                  hl.agg.hist(-hl.log10(mt.variant_qc.p_value_hwe), 0, 10, 40))
)
edges, freq = hwe_hist.bin_edges, hwe_hist.bin_freq
mids = [(a + b) / 2 for a, b in zip(edges, edges[1:])]
width = (edges[1] - edges[0]) * 0.9

fig, ax = plt.subplots(figsize=(9, 4))
# YOUR CODE HERE: bar chart of -log10(HWE p-value) distribution
# Add a vertical dashed red line at x=6 to mark the filter threshold (p=1e-6)


In [ ]:
n_vars_before = mt.count_rows()
mt = mt.filter_rows((mt.variant_qc.AF[1] > 0.01) & (mt.variant_qc.p_value_hwe > 1e-6))
n_vars_after = mt.count_rows()
print('Variants: %d → %d (removed %d, %.1f%%)' % (
    n_vars_before, n_vars_after,
    n_vars_before - n_vars_after,
    100 * (n_vars_before - n_vars_after) / n_vars_before
))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
# YOUR CODE HERE: bar chart comparing n_vars_before vs n_vars_after
# Label bars with the exact counts


In [ ]:
n_variants, n_samples = mt.count()
print('After QC: %d variants x %d samples' % (n_variants, n_samples))

## 4. Association Analysis

### Population Stratification — PCA

The 1000 Genomes dataset spans 5 continental populations (AFR, AMR, EAS, EUR, SAS). We compute **Principal Components (PCs)** to capture ancestry and include them as covariates.

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
pprint(eigenvalues[:5])
mt = mt.annotate_cols(scores=pcs[mt.s].scores)

In [ ]:
# YOUR CODE HERE: scatter plot of PC1 vs PC2, coloured by SuperPopulation
# Hint: use hl.plot.scatter(mt.scores[0], mt.scores[1], label=mt.pheno.SuperPopulation, ...)
# then show(p)


In [ ]:
# YOUR CODE HERE: scatter plot of PC2 vs PC3, coloured by SuperPopulation


### Naive GWAS (no covariates)

In [ ]:
gwas_naive = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0]
)
p = hl.plot.qq(gwas_naive.p_value, title='QQ Plot — naive (no covariates)')
show(p)
p = hl.plot.manhattan(gwas_naive.p_value, title='Manhattan Plot — naive (no covariates)')
show(p)

**Genomic inflation factor (λ)** is shown in the QQ plot:
- λ ≈ 1.0 → well-calibrated
- λ > 1.05 → inflation (likely population stratification)
- λ < 1.0 → deflation

### Corrected GWAS (sex + PC1–3 as covariates)

In [ ]:
# YOUR CODE HERE: run hl.linear_regression_rows with sex and PC1-3 as covariates
# covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]]
# Then plot QQ and Manhattan plots
gwas = None  # replace with your linear_regression_rows call


### Top Hits

In [ ]:
gwas.order_by(gwas.p_value).select('p_value', 'beta', 'standard_error', 't_stat').show(20, width=120)

In [ ]:
top = gwas.order_by(gwas.p_value).take(20)
betas = np.array([r.beta for r in top])
ses   = np.array([r.standard_error for r in top])
labels = ['%s/%s' % (r.locus, r.alleles[1]) for r in top]

fig, ax = plt.subplots(figsize=(8, 7))
y = np.arange(len(top))
ax.errorbar(betas, y, xerr=1.96 * ses, fmt='o', color='steelblue',
            capsize=3, linewidth=1, markersize=5)
ax.axvline(0, color='grey', linestyle='--', linewidth=1)
ax.set_yticks(y)
ax.set_yticklabels(labels, fontsize=8)
ax.set(xlabel='Effect size (beta) ± 95% CI', title='Top 20 Variants by P-value')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

---
**🔍 Exercise 3:** Run a **logistic GWAS** for the binary `PurpleHair` phenotype. Do the top hits overlap with caffeine?

> *Starter code:*
> ```python
> gwas_binary = hl.logistic_regression_rows(
>     test='wald',
>     y=mt.pheno.PurpleHair,
>     x=mt.GT.n_alt_alleles(),
>     covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]]
> )
> ```

In [ ]:
# Exercise 3 — your code here


## Summary

In this tutorial you:

- ✅ Downloaded and imported the 1000 Genomes VCF into a Hail MatrixTable
- ✅ Explored the dataset: MatrixTable structure, population composition, allele frequency spectrum, genotype distribution
- ✅ Applied sample QC (depth, call rate), allelic balance filtering, and variant QC (MAF, HWE)
- ✅ Ran PCA to capture population structure
- ✅ Compared a naive vs. covariate-corrected GWAS, and observed the effect of stratification on λ
- ✅ Identified and visualised the top associated variants